<a href="https://colab.research.google.com/github/Korsholm22/M4_Group_Assignments/blob/main/Group_Assignment_3/SBERT_Netflix_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
# Pip installs
!pip install sentence-transformers -q
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 93.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
from sentence_transformers.util import cos_sim
import gradio as gr

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [3]:
# Importing the dataset
df_netflix = pd.read_csv('https://raw.githubusercontent.com/Korsholm22/M4_Group_Assignments/main/Group_Assignment_3/Data/netflix_titles.csv')

In [4]:
# Examining the dataset
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# Preprocessing and Feature Engineering

In [6]:
# Checking the dataset for NaN values
df_netflix.isna().sum()

show_id          0
type             0
title            0
director        38
cast            11
country         40
date_added       0
release_year     0
rating           0
duration         0
listed_in        0
description      0
dtype: int64

In [8]:
# Merging title, listed_in and description into one column to include more information in the semantic search
df_netflix['information'] = df_netflix['listed_in'].str.cat(df_netflix['description'], sep =". ")
df_netflix['information'] = df_netflix['title'].str.cat(df_netflix['information'], sep =". ")

# Checking if the merge is succesfull
df_netflix['information'][1]

'Blood & Water. International TV Shows, TV Dramas, TV Mysteries. After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.'

In [9]:
# Creating 5 search examples to examine performance of the model
search_examples = ["Action movie taking place in space",
                    "Sad movie where the dog dies",  
                    "Documentray about turtles and plastic straws",       
                    "Funny movie with Kevin Hart and the Rock",    
                    "True crime tv show where the detective turns out to be the villain"]

In [10]:
# Embedding the search examples
embeddings = model.encode(search_examples)

embeddings.shape

(5, 768)

In [11]:
# Converting the column show_information to a list
show_information = df_netflix['information'].values.tolist()

In [12]:
# Checking that the conversion is succesfull
show_information[0:5]

['Blood & Water. International TV Shows, TV Dramas, TV Mysteries. After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.',
 'Ganglands. Crime TV Shows, International TV Shows, TV Action & Adventure. To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.',
 'Kota Factory. International TV Shows, Romantic TV Shows, TV Comedies. In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.',
 'Midnight Mass. TV Dramas, TV Horror, TV Mysteries. The arrival of a charismatic young priest brings glorious miracles, ominous mysteries and renewed religious fervor to a dying town desperate to believe.',
 "My Little Pony: A New Generation. Children & Family Movies. Equestria's divided. But a bright-eyed hero believes Earth Pon

In [13]:
# Embedding show_information and converting it into tensors
corpus_embeddings = model.encode(show_information, convert_to_tensor=True)

# Modelling

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(show_information))
for query in search_examples:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(show_information[idx], "(Score: {:.4f})".format(score))

In [17]:
# Converting columns from the dataset to lists to be used as output in the semantic search function
title = df_netflix['title'].values.tolist()
description = df_netflix['description'].values.tolist()

In [ ]:
# Defining the semantic search function to return Title, description and cosinus score
def query_corpus(query):
  results = []
  query_embedding = model.encode(query, convert_to_tensor=True)
  cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)

  for score, idx in zip(top_results[0], top_results[1]):
    results.append(
        (title[idx], 
         description[idx],
         score))
    Title = "Title: " + results[0][0]
    desc = "Description: " + results[0][1]
    score = "Cosinus Score: " + str(results[0][2].numpy())  
  return Title, desc, score

In [98]:
# Insert the search term for the semantic search function
query_corpus('Naruto')

('Title: Naruto Shippuden the Movie: Blood Prison',
 'Description: Mistakenly accused of an attack on the Fourth Raikage, ninja Naruto is imprisoned in the impenetrable Hozuki Castle and his powers are sealed.',
 'Cosinus Score: 0.26622343')

# Gradio

In [101]:
# Setting up gradio

gr.Interface(fn=query_corpus, inputs=["text"], outputs=["text", "text", "label"]).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>